# Research Methods 
## Bootstraping and randomization

**11. December 2017**

Fabian Karl & Robert Brown

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import scipy.optimize as opt
from random import shuffle

import matplotlib.pyplot as plt

# Bootstraping

In [2]:
df = pd.read_csv('BodyFat.csv', thousands=',')
df.head(15)

,Unnamed: 0,Bodyfat,Age,Weight,Height,Neck,Chest,Abdomen,Ankle,Biceps,Wrist
0,1,32.3,41,247.25,73.50,42.1,117.0,115.6,26.3,37.3,19.7
1,2,22.5,31,177.25,71.50,36.2,101.1,92.4,24.6,30.1,18.2
2,3,22.0,42,156.25,69.00,35.5,97.8,86.0,24.0,31.2,17.4
3,4,12.3,23,154.25,67.75,36.2,93.1,85.2,21.9,32.0,17.1
4,5,20.5,46,177.00,70.00,37.2,99.7,95.6,22.5,29.1,17.7
5,6,22.6,54,198.00,72.00,39.9,107.6,100.0,22.0,35.9,18.9
6,7,28.7,43,200.50,71.50,37.9,107.2,103.1,23.7,32.1,18.7
7,8,21.3,42,163.00,70.25,35.3,93.5,89.6,21.9,30.7,17.4
8,9,29.9,37,241.25,71.50,42.1,119.2,110.3,24.8,34.4,18.4
9,10,21.3,41,218.50,71.00,39.8,111.7,100.5,25.2,37.5,18.7


In [3]:
df.describe()

,Unnamed: 0,Bodyfat,Age,Weight,Height,Neck,Chest,Abdomen,Ankle,Biceps,Wrist
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,50.500000,18.601000,44.880000,177.451500,70.355000,37.894000,100.681000,91.867000,22.900000,32.292000,18.221000
std,29.011492,8.006683,11.417937,29.508013,2.714807,2.302998,8.516493,10.261235,1.346451,3.065141,0.999322
min,1.000000,3.700000,23.000000,127.500000,65.000000,32.800000,83.400000,70.400000,20.100000,25.600000,16.300000
25%,25.750000,12.375000,39.750000,152.937500,68.250000,36.275000,94.000000,83.825000,22.000000,29.900000,17.600000
50%,50.500000,18.950000,44.000000,176.125000,70.000000,37.900000,99.250000,90.150000,22.600000,32.000000,18.200000
75%,75.250000,24.575000,52.000000,195.437500,72.250000,39.400000,105.600000,98.975000,23.725000,34.400000,18.825000
max,100.000000,40.100000,74.000000,262.750000,77.750000,43.200000,128.300000,126.200000,27.000000,38.500000,21.400000


In [4]:
def bootstrap_sampling(df, N = 30, K = 10, f = np.median):
    samples = {}
    for _ in range(K):
        sample = df.sample(N, replace = True).apply(f)
        for i, statistic in enumerate(sample):
            key = sample.keys()[i]
            if key not in samples: 
                samples[key] = []
            samples[key].append(statistic)
    samples = {k:np.array(v) for k, v in samples.items()}
    return samples

In [5]:
K = 500

H_0 = 13.5
alpha = 0.05

In [6]:
bootstrap = bootstrap_sampling(df, N = 30, K = K, f = np.median)['Bodyfat']
shift = H_0 - df['Bodyfat'].median()
bootstrap = bootstrap + shift

In [7]:
calculate_p = lambda v: np.sum((bootstrap < min(v)) | (bootstrap > max(v)))/float(K)
p = calculate_p([H_0 - shift, H_0 + shift])

print 'p-value: {0}'.format(p)

p-value: 0.004


since we have $p < \alpha$ so we reject our null hypothesis $H_0: median = 13.5$. 

Alternatively this can be done direclty with critical values.

In [8]:
ordered = np.sort(bootstrap)
c_index = [int(K*alpha/2),  int(K*(1 - alpha/2))]

print 'critical values of {0} and {1}'.format(*ordered[c_index])
print 'sample median = {0}'.format(df['Bodyfat'].median())

critical values of 9.0 and 16.8
sample median = 18.95


Since our sample median does not fall inside of this range, we reject our null hypothesis  $H_0: median = 13.5$